# LangChain Basics

In [2]:
# installing the required libraries
!pip install -r ./requirements.txt -q

In [ ]:
# !pip - installs the packages in the base environment
# pip - installs the packages in the virtual environment

In [3]:
!pip show langchain

Name: langchain
Version: 0.0.238
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: C:\Users\Ryan\anaconda3\envs\llm_env\Lib\site-packages
Requires: aiohttp, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [5]:
# upgrading langchain
!pip install langchain --upgrade -q

#### Python-dotenv

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

# loading the API Keys (OpenAI, Pinecone) from .env
load_dotenv(find_dotenv(), override=True)

# os.environ.get('PINECONE_API_KEY')

True

### LLM Models (Wrappers): GPT-3

In [28]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=512)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'max_tokens': 512, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


In [29]:
output = llm('explain quantum mechanics in one sentence')
print(output)

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..




Quantum mechanics is a branch of physics that explains the behavior of matter and energy on the atomic and subatomic level.


In [30]:
print(llm.get_num_tokens('explain quantum mechanics in one sentence'))

7


In [31]:
output = llm.generate(['... is the capital of France.', 
                   'What is the formula for the area of a circle?'])

In [32]:
print(output.generations)

[[Generation(text='\n\nParis', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nThe formula for the area of a circle is A = πr², where r is the radius of the circle and π is the mathematical constant approximately equal to 3.14.', generation_info={'finish_reason': 'stop', 'logprobs': None})]]


In [33]:
print(output.generations[0][0].text)



Paris


In [34]:
len(output.generations)

2

In [35]:
output = llm.generate(['Write an orignal tagline for a burger restaurant'] * 3)

In [36]:
for o in output.generations:
    print(o[0].text, end='')



"Taste the Burgers of Your Dreams!"

"Sink your teeth into something delicious - try a burger from [restaurant name] today!"

"Beefed Up Burgers: The Taste That Satisfies!"

### ChatModels: GPT-3.5-Turbo and GPT-4

In [37]:
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [38]:
chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5, max_tokens=1024)
messages = [
    SystemMessage(content='You are a physicist and respond only in German.'),
    HumanMessage(content='explain quantum mechanics in one sentence')
]
output = chat(messages)

In [39]:
print(output.content)

Quantenmechanik beschreibt das Verhalten von Teilchen auf atomarer und subatomarer Ebene durch probabilistische Wellenfunktionen.


### Prompt Templates

In [41]:
from langchain import PromptTemplate

In [42]:
template = '''You are an experienced virologist.
Write a few sentences about the following {virus} in {language}.'''

prompt = PromptTemplate(
    input_variables=['virus', 'language'],
    template=template
)
print(prompt)

input_variables=['virus', 'language'] output_parser=None partial_variables={} template='You are an experienced virologist.\nWrite a few sentences about the following {virus} in {language}.' template_format='f-string' validate_template=True


In [44]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7)
output = llm(prompt.format(virus='Malaria', language='French'))
print(output)



La malaria est une maladie infectieuse causée par des parasites du genre Plasmodium, qui sont transmis à l'être humain par des moustiques infectés. Les symptômes de la malaria comprennent des frissons, de la fièvre, des maux de tête, des douleurs musculaires et des vomissements. La malaria peut être grave et potentiellement mortelle si elle n'est pas traitée rapidement et correctement. Il est important pour les médecins et les virologistes de comprendre le cycle de vie et les caractéristiques des parasites Plasmodium afin de mieux contrôler et traiter la maladie.


### Simple Chains

In [45]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)
template = '''You are an experienced virologist.
Write a few sentences about the following {virus} in {language}.'''

prompt = PromptTemplate(
    input_variables=['virus', 'language'],
    template=template
)

chain = LLMChain(llm=llm, prompt=prompt)
output = chain.run({'virus': 'HSV', 'language': 'french'})


In [46]:
print(output)

Le HSV, ou virus de l'herpès simplex, est un virus à ADN de la famille des Herpesviridae. Il existe deux types de HSV : le HSV-1, principalement responsable des infections buccales et labiales, et le HSV-2, qui est généralement associé aux infections génitales. Ces virus peuvent se transmettre par contact direct avec une personne infectée ou par le biais de fluides corporels. Les symptômes courants comprennent des lésions cutanées douloureuses, des démangeaisons et des brûlures. Bien que les infections à HSV ne puissent pas être guéries, des médicaments antiviraux peuvent aider à réduire les symptômes et à prévenir les récidives.


### Sequential Chains

In [47]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=1024)
prompt1 = PromptTemplate(
    input_variables=['concept'],
    template='''You are an experienced scientist and Python programmer.
    Write a function that implements the concept of {concept}.'''
)
chain1 = LLMChain(llm=llm1, prompt=prompt1)


llm2 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1.2)
prompt2 = PromptTemplate(
    input_variables=['function'],
    template='Given the Python function {function}, describe it as detailed as possible.'
)
chain2 = LLMChain(llm=llm2, prompt=prompt2)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
output = overall_chain.run('softmax')





> Entering new SimpleSequentialChain chain...


def softmax(x):
    """
    Compute softmax values for each sets of scores in x.
    
    Parameters
    ----------
    x : list of numbers, numpy array
        Input scores
 
    Returns
    -------
    output : numpy array
        Softmax scores
    
    """
    # Use exp() to calculate softmax
    exps = np.exp(x)
    # Calculate the sum of the exponentials 
    sum_exps = np.sum(exps)
    # Calculate softmax
    output = exps / sum_exps
    return output
The given function is a Python function named `softmax`. It computes the softmax values for a list of scores provided as input. The function takes in an input variable `x`, which is expected to be a list of numbers or a numpy array.

The softmax function is commonly used in machine learning and deep learning models to convert a vector of real values into a probability distribution. It is calculated by exponentiating each element of the input vector and then normalizing the resulting

### LangChain Agents

In [48]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool  
from langchain.llms import OpenAI

In [49]:
llm = OpenAI(temperature=0)
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)
# agent_executor.run('Calculate the square root of the factorial of 20 \
# and display it with 4 decimal points')

agent_executor.run('what is the answer to 5.1 ** 7.3?')





> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


 I need to use the Python REPL to calculate this
Action: Python_REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought: I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


'146306.05007233328'